In [1]:
import sys
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/analysisTools")
from analysisImports import *
setLibParams()

cfgDir = "/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMeAnalysis/python_analysis/configs/"
plt.style.use(cfgDir+'plt_settings.mplstyle')

In [2]:
a = ak.Array([[1,2],[3,4,5]])
b = ak.argmin(a,axis=1,keepdims=True)
a[b]

<Array [[1], [3]] type='2 * var * ?int64'>

In [14]:
# loading in all signal sample data
importlib.reload(aTools)
az = aTools.Analyzer(cfgDir+'sample_configs/signal_aEM.json',cfgDir+'histo_configs/histoList.json',cfgDir+'histo_configs/histoPlotConfig.json')
fileset = az.sample_locs
samples = az.sample_names
sample_info = az.sample_info
histos = az.histos

In [7]:
eff_table_in = {'Sample':[],'Full Unique Match':[],'Full Match':[],'Any Match':[]}
eff_table_out = {'Sample':[],'Full Unique Match':[],'Full Match':[],'Any Match':[]}
for samp in samples:
    info = sample_info[samp]
    files = fileset[samp]
    eff_table_in['Sample'].append("({0},{1}), {2} cm".format(info["Mchi"],info["dMchi"],info["ctau"]))
    eff_table_out['Sample'].append("({0},{1}), {2} cm".format(info["Mchi"],info["dMchi"],info["ctau"]))
    print("Mchi = {0:.1f}, dMchi = {1:.1f}, ct = {2}".format(info["Mchi"],info["dMchi"],info["ctau"]))
    nEvents = 0
    n_fullUnique_in = 0
    n_fullUnique_out = 0
    n_full_in = 0
    n_full_out = 0
    n_anyMatch_in = 0
    n_anyMatch_out = 0
    n_agree = 0
    for fname in files:
        NanoAODSchema.warn_missing_crossrefs = False
        loc = uproot.open(fname)
        events = NanoEventsFactory.from_root(loc,treepath="ntuples/outT",schemaclass=NanoAODSchema).events()

        conv1_inPos = ak.zip(
            {
                "x":events.Conversion.trk1_innerEta,
                "y":events.Conversion.trk1_innerPhi,
            },
            with_name="TwoVector"
        )
        conv2_inPos = ak.zip(
            {
                "x":events.Conversion.trk2_innerEta,
                "y":events.Conversion.trk2_innerPhi,
            },
            with_name="TwoVector"
        )
        conv1_outPos = ak.zip(
            {
                "x":events.Conversion.trk1_outerEta,
                "y":events.Conversion.trk1_outerPhi,
            },
            with_name="TwoVector"
        )
        conv2_outPos = ak.zip(
            {
                "x":events.Conversion.trk2_outerEta,
                "y":events.Conversion.trk2_outerPhi,
            },
            with_name="TwoVector"
        )
        gen_eles = events.GenPart[events.GenPart.ID == 11]
        gen_pos = events.GenPart[events.GenPart.ID == -11]
        gen_ele_pos = ak.zip(
            {
                "x":gen_eles.eta,
                "y":gen_eles.phi,
            },
            with_name="TwoVector"
        )
        gen_pos_pos = ak.zip(
            {
                "x":gen_pos.eta,
                "y":gen_pos.phi,
            },
            with_name="TwoVector"
        )
        gen_ele_pos = ak.with_name(ak.broadcast_arrays(ak.flatten(gen_ele_pos),conv1_inPos)[0],"TwoVector")
        gen_pos_pos = ak.with_name(ak.broadcast_arrays(ak.flatten(gen_pos_pos),conv1_inPos)[0],"TwoVector")

        d1_e_in = (conv1_inPos - gen_ele_pos).r
        d1_p_in = (conv1_inPos - gen_pos_pos).r
        d2_e_in = (conv2_inPos - gen_ele_pos).r
        d2_p_in = (conv2_inPos - gen_pos_pos).r
        d1_e_in = d1_e_in[ak.argmin(d1_e_in,axis=1,keepdims=True)]
        d1_p_in = d1_p_in[ak.argmin(d1_p_in,axis=1,keepdims=True)]
        d2_e_in = d2_e_in[ak.argmin(d2_e_in,axis=1,keepdims=True)]
        d2_p_in = d2_p_in[ak.argmin(d2_p_in,axis=1,keepdims=True)]
        match_1e2p_in = (d1_e_in < 0.1) & (d2_p_in < 0.1) & (d1_p_in > 0.1) & (d2_e_in > 0.1)
        match_1p2e_in = (d1_p_in < 0.1) & (d2_e_in < 0.1) & (d1_e_in > 0.1) & (d2_p_in > 0.1)
        fullUniqueMatch_in = match_1e2p_in | match_1p2e_in
        fullUniqueMatch_in = ak.fill_none(ak.flatten(fullUniqueMatch_in),False)
        fullMatch_in = ((d1_e_in < 0.1) | (d1_p_in < 0.1)) & ((d2_e_in < 0.1) | (d2_p_in < 0.1))
        fullMatch_in = ak.fill_none(ak.flatten(fullMatch_in),False)
        any_match_in = (d1_e_in < 0.1) | (d1_p_in < 0.1) | (d2_e_in < 0.1) | (d2_p_in < 0.1)
        any_match_in = ak.fill_none(ak.flatten(any_match_in),False)
        

        d1_e_out = (conv1_outPos - gen_ele_pos).r
        d1_p_out = (conv1_outPos - gen_pos_pos).r
        d2_e_out = (conv2_outPos - gen_ele_pos).r
        d2_p_out = (conv2_outPos - gen_pos_pos).r
        d1_e_out = d1_e_out[ak.argmin(d1_e_out,axis=1,keepdims=True)]
        d1_p_out = d1_p_out[ak.argmin(d1_p_out,axis=1,keepdims=True)]
        d2_e_out = d2_e_out[ak.argmin(d2_e_out,axis=1,keepdims=True)]
        d2_p_out = d2_p_out[ak.argmin(d2_p_out,axis=1,keepdims=True)]
        match_1e2p_out = (d1_e_out < 0.1) & (d2_p_out < 0.1) & (d1_p_out > 0.1) & (d2_e_out > 0.1)
        match_1p2e_out = (d1_p_out < 0.1) & (d2_e_out < 0.1) & (d1_e_out > 0.1) & (d2_p_out > 0.1)
        fullUniqueMatch_out = match_1e2p_out | match_1p2e_out
        fullUniqueMatch_out = ak.fill_none(ak.flatten(fullUniqueMatch_out),False)
        fullMatch_out = ((d1_e_out < 0.1) | (d1_p_out < 0.1)) & ((d2_e_out < 0.1) | (d2_p_out < 0.1))
        fullMatch_out = ak.fill_none(ak.flatten(fullMatch_out),False)
        any_match_out = (d1_e_out < 0.1) | (d1_p_out < 0.1) | (d2_e_out < 0.1) | (d2_p_out < 0.1)
        any_match_out = ak.fill_none(ak.flatten(any_match_out),False)

        nEvents += len(events.eventNum)
        n_fullUnique_in += len(events.eventNum[fullUniqueMatch_in])
        n_fullUnique_out += len(events.eventNum[fullUniqueMatch_out])
        n_full_in += len(events.eventNum[fullMatch_in])
        n_full_out += len(events.eventNum[fullMatch_out])
        n_anyMatch_in += len(events.eventNum[any_match_in])
        n_anyMatch_out += len(events.eventNum[any_match_out])
        n_agree += len(events.eventNum[fullUniqueMatch_in == fullUniqueMatch_out])
    eff_table_in['Full Unique Match'].append(100*n_fullUnique_in/nEvents)
    eff_table_out['Full Unique Match'].append(100*n_fullUnique_out/nEvents)
    eff_table_in['Full Match'].append(100*n_full_in/nEvents)
    eff_table_out['Full Match'].append(100*n_full_out/nEvents)
    eff_table_in['Any Match'].append(100*n_anyMatch_in/nEvents)
    eff_table_out['Any Match'].append(100*n_anyMatch_out/nEvents)
    print("Inner Position Full Unique Match : {0:.4f}%".format(100*n_fullUnique_in/nEvents))
    print("Inner Position Any Match : {0:.4f}%".format(100*n_anyMatch_in/nEvents))
    print("Outer Position Full Match : {0:.4f}%".format(100*n_fullUnique_out/nEvents))
    print("Outer Position Any Match : {0:.4f}%".format(100*n_anyMatch_out/nEvents))
    print("In/Out agreement fraction : {0:.4f}%".format(100*n_agree/nEvents))
    print("========================================")

Mchi = 48.0, dMchi = 16.0, ct = 1
Inner Position Full Unique Match : 0.1771%
Inner Position Any Match : 13.5751%
Outer Position Full Match : 0.1685%
Outer Position Any Match : 8.5499%
In/Out agreement fraction : 99.6894%
Mchi = 48.0, dMchi = 16.0, ct = 10
Inner Position Full Unique Match : 0.1728%
Inner Position Any Match : 12.0117%
Outer Position Full Match : 0.1535%
Outer Position Any Match : 7.4875%
In/Out agreement fraction : 99.7094%
Mchi = 48.0, dMchi = 16.0, ct = 100
Inner Position Full Unique Match : 0.1666%
Inner Position Any Match : 7.6514%
Outer Position Full Match : 0.1293%
Outer Position Any Match : 5.1976%
In/Out agreement fraction : 99.7577%
Mchi = 48.0, dMchi = 16.0, ct = 1000
Inner Position Full Unique Match : 0.0472%
Inner Position Any Match : 1.9794%
Outer Position Full Match : 0.0405%
Outer Position Any Match : 1.4754%
In/Out agreement fraction : 99.9338%
Mchi = 42.0, dMchi = 4.0, ct = 1
Inner Position Full Unique Match : 0.0517%
Inner Position Any Match : 2.3629%
O

In [9]:
df_in = pd.DataFrame(eff_table_in)
df_out = pd.DataFrame(eff_table_out)

In [13]:
print(df_in.to_latex(float_format="%.4f",index=False))

\begin{tabular}{lrrr}
\toprule
              Sample &  Full Unique Match &  Full Match &  Any Match \\
\midrule
   (48.0,16.0), 1 cm &             0.1771 &      9.0704 &    13.5751 \\
  (48.0,16.0), 10 cm &             0.1728 &      7.7178 &    12.0117 \\
 (48.0,16.0), 100 cm &             0.1666 &      4.1208 &     7.6514 \\
(48.0,16.0), 1000 cm &             0.0472 &      0.8684 &     1.9794 \\
    (42.0,4.0), 1 cm &             0.0517 &      1.5357 &     2.3629 \\
   (42.0,4.0), 10 cm &             0.0760 &      1.2042 &     1.9919 \\
  (42.0,4.0), 100 cm &             0.0305 &      0.5101 &     1.0415 \\
 (42.0,4.0), 1000 cm &             0.0091 &      0.1004 &     0.2372 \\
\bottomrule
\end{tabular}



In [14]:
print(df_out.to_latex(float_format="%.4f",index=False))

\begin{tabular}{lrrr}
\toprule
              Sample &  Full Unique Match &  Full Match &  Any Match \\
\midrule
   (48.0,16.0), 1 cm &             0.1685 &      3.0858 &     8.5499 \\
  (48.0,16.0), 10 cm &             0.1535 &      2.5692 &     7.4875 \\
 (48.0,16.0), 100 cm &             0.1293 &      1.6379 &     5.1976 \\
(48.0,16.0), 1000 cm &             0.0405 &      0.4307 &     1.4754 \\
    (42.0,4.0), 1 cm &             0.0213 &      0.2798 &     1.1069 \\
   (42.0,4.0), 10 cm &             0.0122 &      0.2159 &     0.9701 \\
  (42.0,4.0), 100 cm &             0.0092 &      0.0916 &     0.5070 \\
 (42.0,4.0), 1000 cm &             0.0000 &      0.0335 &     0.1490 \\
\bottomrule
\end{tabular}

